In [16]:
import json
import pymongo
import pandas as pd
import os
import shutil
from bson import ObjectId
from datetime import datetime
from pathlib import Path

# Define a function to update column 'syncedBySchool' based on the values in column 'status'
def update_column(row):
    if row['status'] == 2:
        return 1  # Set 'syncedBySchool' to 1 when 'status' is 2
    else:
        return 0  # Set 'syncedBySchool' to 0 when 'status' is 1

#datetime(year, month, day)
start_date = datetime(2023, 10, 1)
end_date = datetime(2023, 10, 31)

current_directory = Path.cwd()
configPath = current_directory / "config.json"

#this will configure your download folder
downloadDirectory = current_directory / "data_ssdm"


# Save the DataFrame to a CSV file
if os.path.exists(str(downloadDirectory)):
    shutil.rmtree(str(downloadDirectory))  # Remove the existing directory and its contents
os.makedirs(str(downloadDirectory))

# Read data from the JSON file
with open(configPath, 'r') as jsonfile:
    config = json.load(jsonfile)

#make sure to initiate vpn to production db first
db = config['DATABASE']
DBclient = pymongo.MongoClient(db['connection'])
DBcol = DBclient[db['collection']]
DBdoc = DBcol[db['document']]

#load school detail from config
school = config['JB_DETAIL']
synced = 0
syncedBySchool = 0

for row in school:
    # Define the filter criteria using the ObjectId instance
    filter_criteria = {"schoolID": ObjectId(row['mongoDB_ID']),
                    "created_at": {"$gte": start_date, "$lte": end_date}
                    }

    # Define the projection to exclude specific fields (e.g., "field_to_exclude")
    exclude = {"_id": 0, "tempat":0, "ic":0, "userID":0, "schoolID":0, "method":0, "updated_at":0, "created_at":0, "submissionID":0}
    results = DBdoc.find(filter_criteria, exclude)
    
    #turn query result into dataframe
    df = pd.DataFrame(list(results))
    df['syncedBySchool'] = syncedBySchool
    df['syncedBySchool'] = df.apply(update_column, axis=1)
    df['synced'] = synced
    
    #set all filtered item to inactive (1:active, 2:inactive)
    results = DBdoc.update_many(filter_criteria, {"$set": {"status": 2}})
    
    #do not print to csv if empty
    if df.empty:
        print(row['NAME'] + " has no submissions")
        continue
    
    #output to csv
    df.to_csv(str(downloadDirectory) + "/" + row['NAME'] + ".csv", index=False)
    print(row['NAME'] + " data has been generated")

KV (ERT) AZIZAH data has been generated
KV PERDAGANGAN has no submissions
KV TANJUNG PUTERI has no submissions
MAKTAB SULTAN ABU BAKAR data has been generated
SEKOLAH TUN FATIMAH data has been generated
SJK (CINA) CHEAH FAH has no submissions
SJK (CINA) FOON YEW 1 data has been generated
SJK (CINA) FOON YEW 2 has no submissions
SJK (CINA) FOON YEW 3 has no submissions
SJK (CINA) FOON YEW 4 data has been generated
SJK (CINA) KEMPAS BARU data has been generated
SJK (CINA) KUO KUANG data has been generated
SJK (CINA) KUO KUANG 2 data has been generated
SJK (CINA) MING CHIH data has been generated
SJK (CINA) MING TERK has no submissions
SJK (CINA) PAI TZE data has been generated
SJK (CINA) PEI HWA data has been generated
SJK (CINA) PING MING data has been generated
SJK (CINA) PU SZE has no submissions
SJK (CINA) ST JOSEPH data has been generated
SJK (CINA) TAMPOI data has been generated
SJK (CINA) THAI HONG data has been generated
SJK (CINA) THORBURN data has been generated
SJK (TAMIL) GEL